# **Project Submission Machine Learning Operations (MLOps) - Amazon Reviews Classification**

In [5]:
import pandas as pd
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

In [6]:
#!pip install tfx

## Data Loading

In [7]:
PIPELINE_ROOT = "pipeline"
SCHEMA_PIPELINE_NAME = "mdpraz-tfdv-schema"
SERVING_MODEL_DIR = "serving_model_dir"

In [8]:
df = pd.read_csv('dataset/amazon_review.csv')

In [9]:
df = df.drop(['title'], axis=1)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15001 entries, 0 to 15000
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    15001 non-null  int64 
 1   content  15001 non-null  object
dtypes: int64(1), object(1)
memory usage: 234.5+ KB


In [11]:
data_path = "data"

# Run in Colab
if not os.path.exists(data_path):
    os.makedirs(data_path)

df.to_csv(os.path.join(data_path, "amazon_reviews.csv"), index=False)


In [12]:
DATA_ROOT = "data"

In [13]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Data Ingestion

**Example Gen**

In [14]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=9),
        example_gen_pb2.SplitConfig.Split(name="eval", hash_buckets=1),
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Pada Example Gen pembagian skema dataset untuk proses training dan evaluasi(testing) menjadi 90:10

## Data Validation

**Statistic Gen**

In [15]:
statistics_gen = StatisticsGen(
    examples=example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

StatisticGen berfungsi untuk membuat statistik dari fitur yang akan digunakan dalam pembentukan model nantinya

In [16]:
interactive_context.show(statistics_gen.outputs["statistics"])

**SchemaGen**

In [17]:
schema_gen = SchemaGen(
    statistics=statistics_gen.outputs["statistics"]
)
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [18]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'content',BYTES,required,,-
'label',INT,required,,-


SchemaGen diperlukan untuk mendapatkan informasi terkait dengan fitur yang digunakan dalam kasus ini yaitu **content** dan **label**.

Fitur label berisi terkait dengan hasil review oleh user bernilai baik atau buruk, dengan content merupakan isi dari review ulasan produk amazon tersebut. Pada fitur label bertipe int sedangkan content bertipe string

**ExampleValidator**

In [19]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"],
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

ExampleValidator digunakan untuk mendeteksi apakah terdapat anomali pada data, dengan menggunakan StatisticGen dan SchemaGen yang telah dibuat sebelumnya

In [20]:
interactive_context.show(example_validator.outputs["anomalies"])

Dari hasil ExampleValidator tidak ditemukannya anomali pada data training maupun evaluation sehingga dapat dilakukan proses selanjutnya

## Data Preprocessing

In [21]:
TRANSFORM_MODULE_FILE = "review_transform.py"

In [22]:
%%writefile {TRANSFORM_MODULE_FILE}
import string
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "label"
FEATURE_KEY = "content"


def transformed(key):
    return f"{key}_xf"


def preprocessing_fn(inputs):
    outputs = dict()

    outputs[transformed(FEATURE_KEY)] = tf.strings.lower(
        inputs[FEATURE_KEY]
    )
    outputs[transformed(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)

    return outputs

Writing review_transform.py


Dilakukan proses untuk membuat modul `review_transform.py` yang berfungsi untuk menyimpan proses transform.

In [23]:
transform = Transform(
    examples=example_gen.outputs["examples"],
    schema=schema_gen.outputs["schema"],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [24]:
TRAINER_MODULE_FILE = "review_trainer.py"

Dilakukan proses untuk membuat modul review_trainer.py yang berfungsi untuk menyimpan proses training.

In [25]:
%%writefile {TRAINER_MODULE_FILE}

#Import library yang digunakan
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "label"
FEATURE_KEY = "content"

#Rename feature yang akan ditransformasi
def transformed(key) :
    return key + "_xf"

#Load data yang dicompress dalam TFRecord
def gzip_reader_fn(filenames) :
    return tf.data.TFRecordDataset(filenames, compression_type = 'GZIP')

#Membuat transformed_feature dan batch data
def input_fn(file_pattern, tf_transform_output, num_epochs,
            batch_size=64)->tf.data.Dataset:

            transform_feature_spec = (
                tf_transform_output.transformed_feature_spec().copy()
            )

            dataset = tf.data.experimental.make_batched_features_dataset(
                file_pattern = file_pattern,
                batch_size = batch_size,
                features = transform_feature_spec,
                reader = gzip_reader_fn,
                num_epochs = num_epochs,
                label_key = transformed(LABEL_KEY)
            )
            return dataset

VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

#Tokenisasi dan standarisasi pada data
vectorize_layer = layers.TextVectorization(
    standardize = "lower_and_strip_punctuation",
    max_tokens = VOCAB_SIZE,
    output_mode = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

#Dilakukan proses embedding
embedding_dim = 16

#Membuat model
def model_builder() :
    #Machine Learning Model
    inputs = tf.keras.Input(shape=(1,), name = transformed(FEATURE_KEY), dtype = tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name = "embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(16, activation = "relu")(x)
    x = layers.Dense(8, activation = "relu")(x)
    outputs = layers.Dense(1, activation = "sigmoid")(x)

    model = tf.keras.Model(inputs = inputs, outputs = outputs)

    # Mendefinisikan compile model
    model.compile(
        loss = 'binary_crossentropy',
        optimizer = tf.keras.optimizers.Adam(0.01),
        metrics = [tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model

#Melakukan preprocessing pada raw request data yang nantinya diperlukan pada proses deployment model
def _get_serve_tf_examples_fn(model, tf_transform_output) :
    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples) :
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)
    return serve_tf_examples_fn

#Melakukan proses training pada model
def run_fn(fn_args : FnArgs) -> None :
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq = 'batch'
    )

    # Mendefinisikan Callback untuk EarlyStopping dan ModelCheckpoint
    es = tf.keras.callbacks.EarlyStopping(monitor = 'val_binary_accuracy', mode = 'max', verbose = 1, patience = 10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor = 'val_binary_accuracy', mode = 'max', verbose=1, save_best_only=True)

    #Load output dari transform data yang telah dibuat
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed(FEATURE_KEY)]
                for i in list(train_set)]
        ]
    )

    #Build model
    model = model_builder()

    #Fitting model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000,
            validation_steps = 1000,
            epochs = 10
    )

    signatures = {
        'serving_default' :
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                tf.TensorSpec(
                                    shape = [None],
                                    dtype = tf.string,
                                    name = 'examples'
                                )
        )
    }

    #Model yang telah dibuat dibuat akan disimpan pada serving_model_dir
    model.save(fn_args.serving_model_dir, save_format = 'tf', signatures = signatures)

Writing review_trainer.py


In [26]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file = os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    train_args = trainer_pb2.TrainArgs(splits = ['train']),
    eval_args = trainer_pb2.EvalArgs(splits = ['eval'])
)

interactive_context.run(trainer)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 content_xf (InputLayer)     [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 16)                272   


Epoch 1: val_binary_accuracy improved from -inf to 0.81165, saving model to pipeline/Trainer/model/6/Format-Serving
1000/1000 [==============================] - 25s 21ms/step - loss: 0.2628 - binary_accuracy: 0.8816 - val_loss: 0.6671 - val_binary_accuracy: 0.8117
Epoch 2/10
 998/1000 [============================>.] - ETA: 0s - loss: 0.0864 - binary_accuracy: 0.9624

1000/1000 [==============================] - 10s 10ms/step - loss: 0.0864 - binary_accuracy: 0.9624
Epoch 3/10
 108/1000 [==>...........................] - ETA: 7s - loss: 0.0647 - binary_accuracy: 0.9711

1000/1000 [==============================] - 1s 1ms/step - loss: 0.0649 - binary_accuracy: 0.9713


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

**Resolver**

In [27]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

resolver = Resolver(
    strategy_class = LatestBlessedModelStrategy,
    model = Channel(type = Model),
    model_blessing = Channel(type = ModelBlessing)
).with_id('Latest_blessed_model_resolver')


In [28]:
interactive_context.run(resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Resolver berfungsi apabila kita ingin membandingkan model yang baru kita buat dengan model versi sebelumnya dengan menyediakan baseline dari model yang digunakan sebagai perbandingan

**Evaluator**

In [29]:
import tensorflow_model_analysis as tfma

config_evaluation = tfma.EvalConfig(
    model_specs = [tfma.ModelSpec(label_key = 'label')],
    slicing_specs = [tfma.SlicingSpec()],
    metrics_specs = [tfma.MetricsSpec(metrics = [tfma.MetricConfig(class_name = 'ExampleCount'),
                tfma.MetricConfig(class_name = 'AUC'),
                tfma.MetricConfig(class_name = 'FalsePositives'),
                tfma.MetricConfig(class_name = 'TruePositives'),
                tfma.MetricConfig(class_name = 'FalseNegatives'),
                tfma.MetricConfig(class_name = 'TrueNegatives'),
                tfma.MetricConfig(class_name = 'BinaryAccuracy',
                    threshold = tfma.MetricThreshold(
                        value_threshold = tfma.GenericValueThreshold(
                            lower_bound = {'value' : 0.5}
                    ),
                    change_threshold = tfma.GenericChangeThreshold(
                        direction = tfma.MetricDirection.HIGHER_IS_BETTER,
                            absolute = {'value' : 0.0001}
                    )
                )
            )
        ])
    ]
)

In [31]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples = example_gen.outputs['examples'],
    model = trainer.outputs['model'],
    baseline_model = resolver.outputs['model'],
    eval_config = config_evaluation
)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

**Evaluator** berisi inputan dari ExampleGen yang telah dibuat sebelumnya, model yang dihasilkan trainer, baseline, dan evaluasi dari model

In [32]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

Merupakan visualisasi dari Evaluator

## Deployment

In [33]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

model_pusher = Pusher(model = trainer.outputs['model'],
    model_blessing = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(filesystem = pusher_pb2.PushDestination.Filesystem(base_directory = "serving_model_dir/amazon-review-model"
        )
    )
)

In [34]:
interactive_context.run(model_pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [35]:
!zip -r /content/pipeline.zip /content/pipeline

  adding: content/pipeline/ (stored 0%)
  adding: content/pipeline/Evaluator/ (stored 0%)
  adding: content/pipeline/Evaluator/blessing/ (stored 0%)
  adding: content/pipeline/Evaluator/blessing/8/ (stored 0%)
  adding: content/pipeline/Evaluator/blessing/8/BLESSED (stored 0%)
  adding: content/pipeline/Evaluator/evaluation/ (stored 0%)
  adding: content/pipeline/Evaluator/evaluation/8/ (stored 0%)
  adding: content/pipeline/Evaluator/evaluation/8/eval_config.json (deflated 67%)
  adding: content/pipeline/Evaluator/evaluation/8/validations.tfrecord (deflated 8%)
  adding: content/pipeline/Evaluator/evaluation/8/plots-00000-of-00001.tfrecord (deflated 17%)
  adding: content/pipeline/Evaluator/evaluation/8/attributions-00000-of-00001.tfrecord (deflated 17%)
  adding: content/pipeline/Evaluator/evaluation/8/metrics-00000-of-00001.tfrecord (deflated 43%)
  adding: content/pipeline/Pusher/ (stored 0%)
  adding: content/pipeline/Pusher/pushed_model/ (stored 0%)
  adding: content/pipeline/Pus

In [36]:
!pip freeze >> requirements.txt

In [37]:
!zip -r /content/serving_model_dir.zip /content/serving_model_dir/

  adding: content/serving_model_dir/ (stored 0%)
  adding: content/serving_model_dir/amazon-review-model/ (stored 0%)
  adding: content/serving_model_dir/amazon-review-model/1708672350/ (stored 0%)
  adding: content/serving_model_dir/amazon-review-model/1708672350/variables/ (stored 0%)
  adding: content/serving_model_dir/amazon-review-model/1708672350/variables/variables.index (deflated 59%)
  adding: content/serving_model_dir/amazon-review-model/1708672350/variables/variables.data-00000-of-00001 (deflated 9%)
  adding: content/serving_model_dir/amazon-review-model/1708672350/saved_model.pb (deflated 77%)
  adding: content/serving_model_dir/amazon-review-model/1708672350/keras_metadata.pb (deflated 85%)
  adding: content/serving_model_dir/amazon-review-model/1708672350/assets/ (stored 0%)
  adding: content/serving_model_dir/amazon-review-model/1708672350/fingerprint.pb (stored 0%)
